In [1]:
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [2]:
import econsir as es

### Simulate

In [3]:
par = es.load_args('params/default.toml')
pol = {'zb': 0.85, 'zc': 50.0}
sim = es.simulate_path(par, pol)
ch = es.outcome_summary(sim)
ch.save('output/simul.svg')
ch

alt.VConcatChart(...)

### Estimate

In [4]:
dat = es.load_data('data/usa.csv')
T0 = len(dat)

In [5]:
es.outcome_summary(dat)

alt.VConcatChart(...)

In [6]:
hard = {'ρ': 0.15}
par_est = es.estimate(dat, par, hard_params=hard, R=500, per=100)

[    0] -22.8506: β=0.4950, γ=0.1010, λ=0.0495, δ=0.0707, κ=0.0005, ψ=505.0250, σ=0.3030, zi=1.5151, p0=0.0991, pr=0.0099, i0=0.9901, β0=1.0000, βr=0.0000, ψ0=1.0000, ψr=0.0000, σc=25.2513, σd=1.0100, σa=10.1005, σo=10.1005, zb1=0.8080, zb2=0.9090, zb3=1.0100, ρ=0.1500
[  100] -10.6488: β=0.4741, γ=0.1055, λ=0.0396, δ=0.0603, κ=0.0009, ψ=351.7772, σ=0.3164, zi=1.5820, p0=0.0389, pr=0.0050, i0=0.3642, β0=1.0000, βr=0.0000, ψ0=1.0000, ψr=0.0000, σc=27.4479, σd=1.4127, σa=10.1636, σo=14.7658, zb1=0.8437, zb2=0.9492, zb3=0.9624, ρ=0.1500
[  200] -9.9514: β=0.4380, γ=0.1076, λ=0.0404, δ=0.0568, κ=0.0008, ψ=315.4977, σ=0.3102, zi=1.6140, p0=0.0147, pr=0.0048, i0=0.1608, β0=1.0000, βr=0.0000, ψ0=1.0000, ψr=0.0000, σc=26.1801, σd=1.1680, σa=7.3578, σo=12.8194, zb1=0.8271, zb2=0.9305, zb3=0.9435, ρ=0.1500
[  300] -10.0600: β=0.4385, γ=0.1074, λ=0.0403, δ=0.0567, κ=0.0008, ψ=314.9193, σ=0.3095, zi=1.6108, p0=0.0056, pr=0.0048, i0=0.1594, β0=1.0000, βr=0.0000, ψ0=1.0000, ψr=0.0000, σc=25.8008, σd

In [7]:
es.save_args(par_est, 'params/estim.toml')

In [8]:
sim_est = es.simulate_path(par_est, T=T0)
es.outcome_summary(sim_est)

alt.VConcatChart(...)

### Policy

In [9]:
T = 365
st_vars = ['s', 'a', 'i', 'r', 'd', 'c', 'ka', 'ki', 'e', 't']
last = sim_est.iloc[-1]
st1 = {k: last[k] for k in st_vars}
opt_date = sim_est.index[-1]

In [10]:
hard_pol = {'kz': 100.0, 'vx': 0.0}

In [11]:
pol_opt = es.optimal_policy(pol, par_est, st1, T, hard_policy=hard_pol, long_run=True)

[    0] 0.9868: zb=0.8585, zc=50.5003, kz=100.0000, vx=0.0000
[  100] 0.9932: zb=0.9995, zc=132.6544, kz=100.0000, vx=0.0000
[  200] 0.9933: zb=1.0169, zc=156.9489, kz=100.0000, vx=0.0000
[  300] 0.9933: zb=1.0175, zc=156.9196, kz=100.0000, vx=0.0000
[  400] 0.9933: zb=1.0171, zc=156.9857, kz=100.0000, vx=0.0000
[  500] 0.9933: zb=1.0175, zc=156.9141, kz=100.0000, vx=0.0000


In [12]:
print(es.eval_policy(hard_pol, par_est, st1, 4*T))
print(es.eval_policy(pol_opt, par_est, st1, T))

0.9743212
0.9933283


In [13]:
sim_opt = es.simulate_path(par_est, pol_opt, st1, T, date=opt_date)

In [14]:
es.outcome_summary(sim_opt)

alt.VConcatChart(...)